https://www.kaggle.com/c/classification-choose-tutors/

In this competition your task will be to predict the probability for a tutor to be a proper one for preparing for the math exam. You will be given two datasets: train.csv (contains all features and the target) and test.csv (only features).

Ваша задача этом соревновании - предсказать вероятность того, подойдет ли репетитор для подготовки к экзамену по математике. Вам будут даны два датасета: train.csv (содержит признаки и целевую переменную) и test.csv (только признаки).

In [18]:
# You can only use these imports:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#from imblearn import over_sampling, under_sampling
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('Choose_tutors_classification_data/train.csv')
test = pd.read_csv('Choose_tutors_classification_data/test.csv')
train.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   years_of_experience  10000 non-null  float64
 3   lesson_price         10000 non-null  float64
 4   qualification        10000 non-null  float64
 5   physics              10000 non-null  float64
 6   chemistry            10000 non-null  float64
 7   biology              10000 non-null  float64
 8   english              10000 non-null  float64
 9   geography            10000 non-null  float64
 10  history              10000 non-null  float64
 11  mean_exam_points     10000 non-null  float64
 12  choose               10000 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 1015.8 KB


In [14]:
test.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,10000,32.0,2.0,2700.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0
1,10001,35.0,6.0,1800.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,71.0
2,10002,44.0,2.0,1200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0
3,10003,44.0,4.0,2950.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,92.0
4,10004,38.0,3.0,1400.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   years_of_experience  10000 non-null  float64
 3   lesson_price         10000 non-null  float64
 4   qualification        10000 non-null  float64
 5   physics              10000 non-null  float64
 6   chemistry            10000 non-null  float64
 7   biology              10000 non-null  float64
 8   english              10000 non-null  float64
 9   geography            10000 non-null  float64
 10  history              10000 non-null  float64
 11  mean_exam_points     10000 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 937.6 KB


Разделим выборку на обучающую и тестовую в соотношении 75/25.

In [19]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train.drop('choose', axis=1),
                                                                    train['choose'], test_size=0.25)

Используем евклидову метрику. Реализуем функцию для ее подсчета.

In [9]:
def e_metrics(x1, x2):
    distance = np.sum(np.square(x1 - x2))
    return np.sqrt(distance)

In [11]:
# функция точности
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [32]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1

        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [25]:
X_train

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
7202,7202,45.0,3.0,2550.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,77.0
6505,6505,41.0,3.0,1500.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,64.0
4883,4883,42.0,4.0,2300.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,84.0
2466,2466,49.0,5.0,2350.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,69.0
8299,8299,57.0,0.0,2200.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2215,2215,38.0,2.0,1600.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,64.0
3060,3060,38.0,5.0,1850.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,87.0
7409,7409,44.0,1.0,1300.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,71.0
285,285,52.0,0.0,1500.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0


In [33]:
k = 10

%time 
y_pred_train = knn(X_train, y_train, X_train, k)
%time 
y_pred = knn(X_train, y_train, X_test, k)

print(f'Точность алгоритма на трейне при k = {k}: {accuracy(y_pred_train, y_train):.3f}')
print(f'Точность алгоритма на тесте при k = {k}: {accuracy(y_pred, y_test):.3f}')

Wall time: 0 ns


KeyError: 0